In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')
FOLDERNAME = 'CS224V Navigator'
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))\

%cd /content/drive/My\ Drive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/CS224V Navigator


In [ ]:
%ls

ColBERT/  experiments/  indexer.ipynb  processed_data/  small_data/


In [ ]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')


Already up to date.


In [ ]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

Obtaining file:///content/drive/MyDrive/CS224V%20Navigator/ColBERT
  Preparing metadata (setup.py) ... done
  Attempting uninstall: colbert
    Found existing installation: colbert 0.2.0
    Uninstalling colbert-0.2.0:
      Successfully uninstalled colbert-0.2.0
  Running setup.py develop for colbert


In [ ]:
import colbert

In [ ]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

We will use the dev set of the **LoTTE benchmark** we recently introduced in the ColBERTv2 paper. We'll download it from HuggingFace datasets. The dev and test sets contain several domain-specific corpora, and we'll use the smallest dev set corpus, namely lifestyle:dev.

For the purposes of a quick demo, we will only run the `Indexer` on the first 10,000 passages. As we do this, let's also remove the queries whose relevant passages are all outside this small set of passages.

In [ ]:
import os
collection_resumes = []

for file_name in os.listdir("processed_data"):
  if True:
    with open(f"processed_data/{file_name}", "r") as file:
      collection_resumes.append(file.read())

print(len(collection_resumes))

2484


## Indexing

For an efficient search, we can pre-compute the ColBERT representation of each passage and index them.

Below, the `Indexer` take a model checkpoint and writes a (compressed) index to disk. We then prepare a `Searcher` for retrieval from this index.

In [ ]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens
max_passages = 10000 # We are well beneath this cap anyways

index_name = f'resume_full_dataset.all.{nbits}bits'

Now run the `Indexer` on the collection subset. Assuming the use of only one GPU, this cell should take about six minutes to finish running.

In [ ]:
checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection_resumes, overwrite=True)



[Nov 06, 20:10:02] #> Creating directory /content/drive/MyDrive/CS224V Navigator/experiments/notebook/indexes/resume_full_dataset.all.2bits 


#> Starting...
#> Joined...


In [ ]:
indexer.get_index() # You can get the absolute path of the index, if needed.

'/content/drive/MyDrive/CS224V Navigator/experiments/notebook/indexes/resume_full_dataset.all.2bits'

## Search

Having built the index and prepared our `searcher`, we can search for individual query strings.

We can use the `queries` set we loaded earlier — or you can supply your own questions. Feel free to get creative! But keep in mind this set of ~300k lifestyle passages can only answer a small, focused set of questions!

In [ ]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection_resumes)


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

[Nov 06, 20:46:32] #> Loading codec...
[Nov 06, 20:46:32] #> Loading IVF...
[Nov 06, 20:46:32] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 484.61it/s]

[Nov 06, 20:46:32] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 17.67it/s]


In [ ]:
query = "which accountants have experience working with foreign companies" # try with an in-range query or supply your own
print(f"#> {query}")

# Find the top-3 passages for this query
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    best_passage = searcher.collection[passage_id]
    break

#> which accountants have experience working with foreign companies


In [ ]:
best_passage

"GENERAL ACCOUNTANT\nProfessional Summary\nI have around 6 years experiences in the field of Accounting & banking works, with Egyptians and foreign companies, I'm really honored to\nintroduce myself as a hardworking, good Communication skills, with the highest level of integrity, honesty, loyalty, strong beliefs, goal orientated,\nsober habits and the desire to produce the best of the work.\nAreas of Expertise\nAbdul Aziz Abdul Ghafar Ahmed) Final accounts and Balance Sheet Ability to deal with the accounting program (QuickBooks). Ability to make\nthe financial statements, accounting cycle by excel. Skill with Microsoft word. Skillfulness with computer.\nWork Experience\n06/2014 to 10/2016\nCompany Name\nCompletion of opening customer account using National bank of Egypt system.\nChecking requests for opening an account and make sure it's compatible with the Egyptian standards for opening an account.\nServing 50 branches per day and complete nearly 500 accounts per day.\nCustomer Servi

In [ ]:
!pip install together

In [ ]:
import together
together.api_key = "<INSERT_API_KEY_HERE>"

search_prompt = f"""
You are acting as an agent that will search this document for the answer to a question. Given the passage and question, you must give the answer and citation

Passage: "{best_passage}"
Question: {query}
Response:
"""

output = together.Complete.create(
    prompt = search_prompt,
    model = 'togethercomputer/llama-2-70b',
    max_tokens = 256,
    temperature = 1,
    top_k = 50,
    top_p = 0.7,
    repetition_penalty = 0,
)


In [ ]:
print(search_prompt)



You are acting as an agent that will search this document for the answer to a question. Given the passage and question, you must give the answer and citation

Passage: "GENERAL ACCOUNTANT
Professional Summary
I have around 6 years experiences in the field of Accounting & banking works, with Egyptians and foreign companies, I'm really honored to
introduce myself as a hardworking, good Communication skills, with the highest level of integrity, honesty, loyalty, strong beliefs, goal orientated,
sober habits and the desire to produce the best of the work.
Areas of Expertise
Abdul Aziz Abdul Ghafar Ahmed) Final accounts and Balance Sheet Ability to deal with the accounting program (QuickBooks). Ability to make
the financial statements, accounting cycle by excel. Skill with Microsoft word. Skillfulness with computer.
Work Experience
06/2014 to 10/2016
Company Name
Completion of opening customer account using National bank of Egypt system.
Checking requests for opening an account and make su

In [ ]:
response = output['output']['choices'][0]['text']
print(response)


```
GENERAL ACCOUNTANT
Professional Summary
I have around 6 years experiences in the field of Accounting & banking works, with Egyptians and foreign companies, I'm really honored to
introduce myself as a hardworking, good Communication skills, with the highest level of integrity, honesty, loyalty, strong beliefs, goal orientated,
sober habits and the desire to produce the best of the work.
Areas of Expertise
Abdul Aziz Abdul Ghafar Ahmed) Final accounts and Balance Sheet Ability to deal with the accounting program (QuickBooks). Ability to make
the financial statements, accounting cycle by excel. Skill with Microsoft word. Skillfulness with computer.
Work Experience
06/2014 to 10/2016
Company Name
Completion of opening customer account using National bank of Egypt system.
Checking requests for opening an account and make sure it's compatible with the Egyptian standards for opening an account.
Serving 50 branches per day and complete nearly 500 accounts per day.
Customer Service
